Import del dataset e definizione funzioni

In [1]:
def name_s(string):
    # sets the surname, name into SURNAME N. and handles errors
    if ',' in string:
        comma = string.index(',');
        risposta = (string[0: comma] + ' ' + string[comma +2] + '.');
    else:
        risposta = string;
    if risposta.upper() == "NAN" or risposta.upper() == "AA. V.":
        risposta = "AA. VV.";
    return risposta;

def name_surname(string):
    if ',' in string:
        comma = string.index(',');
        risposta = (string[comma +2:] + ' ' + string[0: comma]);
    else:
        risposta = string;
    if risposta.upper() == "NAN" or risposta.upper() == "AA. V.":
        risposta = "AA. VV.";
    return risposta;

def clean_name(string, upper = 1):
    # splits string into different and adds commas
    tot = "";
    if string == "NESSUNO":
        return "";        
    if upper:
        for i in string.split(";"):
            if tot != "":
                tot = tot + ", ";
            tot = tot + name_s(i);       
        return tot.upper() + ", ";
    else:
        for i in string.split(";"):
            if tot != "":
                tot = tot + " e ";
            tot = tot + name_surname(i);
        return tot + ", ";
    
def clean_pages(string):
    # adds prefix to pages
    if '-' in str(string):
        return ', pp. ' + str(string);
    else:
        return ', p. ' + str(string);

def italic(string):
    # turns to italic for html
    return '<em>' + string + '</em>'

def save_row(string, file):
    # appends to file "file" string "string"
    out_file = open(file,'a')
    out_file.write(string)
    out_file.write('<br>')
    out_file.close()

def save_html(lista, file = 'output.html'):
    # creates and closes files, uses "save row" for each row
    
    output_file = open(file,'w')
    output_file.write('')
    output_file.close()

    for i in lista:
        save_row(i, file);

    output_file = open(file,'a')
    output_file.write('')
    output_file.close()
    
def section_lista(string, lista):
    # adds formatting to export
    lista.append('--------------')
    lista.append(string)
    lista.append('--------------')
    return lista;

In [2]:
import pandas as pd
df = pd.read_csv('bib.csv')
df.sort_values(by=['Author'], inplace=True) # keeps sorting

Definizione dei vari subset e controllo numeriche

In [3]:
article_journal =  df[  (df['Item Type'] == 'journalArticle') & ( df['Url'].isnull() & df['DOI'].isnull())]
#article_journal_url =  df[  (df['Item Type'] == 'journalArticle') & df['Url'].notnull()]
article_journal_url =  df[  (df['Item Type'] == 'journalArticle') & ( df['Url'].notnull() | df['DOI'].notnull())]

article_newspaper = df[  (df['Item Type'] == 'newspaperArticle') & df['Url'].isnull()]
article_newspaper_url = df[  (df['Item Type'] == 'newspaperArticle') & df['Url'].notnull()]

book = df[( df['Item Type'] == 'book' ) & df['Editor'].isnull()]
book_editor = df[( df['Item Type'] == 'book' ) & df['Editor'].notnull()]

book_section = df[( df['Item Type'] == 'bookSection') & df['Editor'].isnull()]
book_section_editor = df[( df['Item Type'] == 'bookSection') & df['Editor'].notnull()]

print('Article Journal - ', len(article_journal.index))
print('Article Journal URL - ', len(article_journal_url.index))
print('Newspaper Journal - ', len(article_newspaper.index))
print('Newspaper Journal URL - ', len(article_newspaper_url.index))
print('Book - ', len(book.index))
print('Book Editor - ', len(book_editor.index))
print('Book Section - ', len(book_section.index))
print('')
print('Total sum - ', len(article_journal.index)+len(article_journal_url.index)+len(article_newspaper.index)+
     len(article_newspaper_url.index)+ len(book.index) + len(book_editor.index) + len(book_section.index))
print('Total df - ', len(df.index))

Article Journal -  46
Article Journal URL -  8
Newspaper Journal -  2
Newspaper Journal URL -  1
Book -  76
Book Editor -  47
Book Section -  6

Total sum -  186
Total df -  203


Articoli di Giornale

In [4]:
# initializes all needed variables
lista_all = [];
numero_entries = 0;

In [5]:
section_lista('article_newspaper', lista_all)

for index, row in article_newspaper.iterrows():
    author = clean_name(row['Author']);
    title = italic(row['Title']);
    pub_title = row['Publication Title'];
    date = (row['Date']);
    if pd.isnull(row['Pages']):
        pages = '';
    else:
        pages = clean_pages(row['Pages']);
    
    esimo = author + title + ' in «' + pub_title + '», ' + date + pages;
    lista_all.append(esimo);
    numero_entries = numero_entries + 1;

In [6]:
section_lista('article_journal', lista_all)

for index, row in article_journal.iterrows():
    author = clean_name(str(row['Author']));
    title = italic(row['Title']);
    pub_title = str(row['Publication Title']);
    date = str(row['Date']);
    if pd.isnull(row['Pages']):
        pages = '';
    else:
        pages = clean_pages(row['Pages']);
    
    volissue = "";
    volume = str(row['Volume']);
    if volume != "nan":
        volissue = volissue + volume + ', ';
    issue = str(row['Issue']);
    if issue != "nan":
        volissue = volissue + issue + ', ';
    
    esimo = author + title + ' in «' + pub_title + '», ' + volissue + date + pages;
    lista_all.append(esimo);
    numero_entries = numero_entries + 1;

In [7]:
section_lista('book', lista_all)

for index, row in book.iterrows():
    author = clean_name(str((row['Author'])));
    title = italic(str(row['Title']));
    publisher = str(row['Publisher']);
    place = str(row['Place']);
    date = str(row['Date']);
    
    esimo = author + title + ', ' + publisher + ', ' + place + ', ' + date;
    lista_all.append(esimo);
    numero_entries = numero_entries + 1;

In [8]:
section_lista('book_editor', lista_all)
    
for index, row in book_editor.iterrows():
    author = clean_name(str((row['Author'])));
    title = italic(str(row['Title']));
    editor = str(clean_name(row['Editor'], 0));
    publisher = str(row['Publisher']);
    place = str(row['Place']);
    date = str(row['Date']);
    
    esimo = author + title + ', a cura di ' + editor + publisher + ', ' + place + ', ' + date;
    lista_all.append(esimo);
    numero_entries = numero_entries + 1;

In [9]:
section_lista('book_section', lista_all)

for index, row in book_section.iterrows():
    author = clean_name(str((row['Author'])));
    title = italic(str(row['Title']));
    pub_title = italic(str(row['Publication Title']));
    pub_editor = clean_name(str(row['Editor']), 0);
    publisher = str(row['Publisher']);
    place = str(row['Place']);
    date = str(row['Date']);
    if pd.isnull(row['Pages']):
        pages = '';
    else:
        pages = clean_pages(row['Pages']);
    
    esimo = author + title + ' in ' + pub_editor + pub_title + ', ' + publisher + ', ' + place + ', ' + date + pages;
    lista_all.append(esimo)
    numero_entries = numero_entries + 1;

In [10]:
section_lista('book_section_editor', lista_all)
    
for index, row in book_section_editor.iterrows():
    author = clean_name(str((row['Author'])));
    title = italic(str(row['Title']));
    editor = str(clean_name(row['Editor'], 0));
    publisher = str(row['Publisher']);
    pub_title = italic(str(row['Publication Title']));
    pub_editor = clean_name(str(row['Editor']), 0);
    place = str(row['Place']);
    date = str(row['Date']);
    if pd.isnull(row['Pages']):
        pages = '';
    else:
        pages = clean_pages(row['Pages']);

    esimo = author + title + ', in <<< INSERIRE >>> ' + pub_title + ', a cura di ' + editor + publisher + ', ' + place + ', ' + date + pages;
    lista_all.append(esimo);
    numero_entries = numero_entries + 1;

In [11]:
section_lista('article journal url', lista_all)

for index, row in article_journal_url.iterrows():
    author = clean_name(str((row['Author'])));
    title = italic(str(row['Title']));
    if str(row['DOI']) != "nan":
        indirizzo = "DOI: " + str(row['DOI']);
    else:
        indirizzo = "URL: " + str(row['Url']);
    
    
    esimo = author + title + ', ' + indirizzo;
    lista_all.append(esimo)
    numero_entries = numero_entries + 1;

In [12]:
section_lista('article_newspaper_url', lista_all)

for index, row in article_newspaper_url.iterrows():
    author = clean_name(str((row['Author'])));
    title = italic(str(row['Title']));
    if str(row['DOI']) != "nan":
        indirizzo = "DOI: " + str(row['DOI']);
    else:
        indirizzo = "URL: " + str(row['Url']);
    
    
    esimo = author + title + ', ' + indirizzo;
    lista_all.append(esimo)
    numero_entries = numero_entries + 1;

In [13]:
section_lista('Modificare a mano', lista_all)
lista_all.append('Gennaro, Montanile, Paulhan - aggiungere curatore')
lista_all.append('Controllare date estese')
lista_all.append('Aggiungere traduttore a Benjamin --> "titolo, traduzione a cura di XXX, "')
lista_all.append('Eva ultima --> modificare a mano con "prefazione di"')
lista_all.append('Aggiungere ebook a "Masino" + "Grignani"')
lista_all.append('Se due nomi consecutivi uguali inserire "-"')
lista_all.append('Inserire autore volumi + curatore "<<< INSERIRE >>>"')
lista_all.append('Modificare Contorbia, Saponaro, Striano')
lista_all.append('Controllare Tortora')
lista_all.append('Risistemare ordine alfabetico')

controllo = 'Censiti ' + str(numero_entries) + ' di un totale ' + str(len(df.index));

section_lista(str(controllo), lista_all)

save_html(lista_all, 'bibliografia.html')

print(controllo)

Censiti 203 di un totale 203
